In [6]:
# imports
import csv 
import pandas as pd 
import os
import requests 
import time
import logging
import json


In [13]:
# constants
# gene/disease qualifiers
# querying for subject disease decreased severity
# looking object gene decreased activity
list_gene_qualifiers = [
{"qualifier_type_id": "object_aspect_qualifier", "qualifier_value": "activity"},
{"qualifier_type_id": "object_direction_qualifier", "qualifier_value": "decreased"},
{"qualifier_type_id": "subject_aspect_qualifier", "qualifier_value": "severity"},
{"qualifier_type_id": "subject_direction_qualifier", "qualifier_value": "decreased"}    
]
# molecular qualifiers

# trapi constants
type_gene = 'biolink:Gene'
type_disease = 'biolink:Disease'
type_molecule = 'biolink:SmallMolecule'
type_predicate = 'biolink:related_to'

# servers
url_genepro = "https://translator.broadinstitute.org/genetics_provider/trapi/v1.3/query"

In [18]:
# variables
# disease 
curie_disease = 'MONDO:0021765' # radiculitis
curie_disease = 'MONDO:0004975' # alzheimer's


In [16]:
# methods - trapi
def get_nodes_one_hop(url, list_source, list_target, list_source_categories, list_target_categories, list_predicates, log=False):
    ''' method to query a trapi url and get the resulting node list back '''
    list_result = []

    # query
    json_response = query_one_hop(url, list_source, list_target, list_source_categories, list_target_categories, list_predicates, log)

    # loop and build the list
    list_nodes = json_response.get("message").get("knowledge_graph").get("nodes")
    if list_nodes and len(list_nodes) > 1:
        for key, value in list_nodes.items():
            list_result.append((key, value.get("name")))

    # log
    if log:
        print("got {} resulting nodes: {}".format(len(list_result), list_result))

    # return
    return list_result

def query_one_hop(url, list_source, list_target, list_source_categories, list_target_categories, list_predicates, log=False):
    ''' method to call a trapi url '''
    response = None

    # build the payload
    payload = build_one_hop_payload(list_source, list_target, list_source_categories, list_target_categories, list_predicates, log=log)

    # call the url
    print("query: {}".format(url))
    response = requests.post(url, json=payload)
    output_json = response.json()
    print("got results from: {}".format(url))

    # log
    # if log:
    #     logger.info("got response: {}".format(output_json))

    # return the json
    return output_json

def build_one_hop_payload(list_source, list_target, list_source_categories, list_target_categories, list_predicates, log=False):
    ''' method to build a one hop json payload for a trapi query '''
    payload = {}

    # build the payload
    nodes = {"n00": build_trapi_query_node(list_source, list_source_categories, log=True), "n01": build_trapi_query_node(list_target, list_target_categories, log=True)}
    edge = {"subject": "n00", "object": "n01"}
    if list_predicates and len(list_predicates) > 0:
        edge["predicates"]= list_predicates
    edges = {"e00": edge}
    payload["message"] = {"query_graph": {"edges": edges, "nodes": nodes}}

    # log
    if log:
        print("build trapi payload: \n{}".format(json.dumps(payload, indent=4)))

    # return
    return payload

def build_trapi_query_node(list_source, list_source_categories, log=False):
    ''' method to build a trapi query node '''
    node = {}

    # log
    # if log:
    #     logger.info("got id: {} and categories: {}".format(list_source, list_source_categories))

    # build the node
    if list_source and len(list_source) > 0:
        node['ids'] = list_source
    if list_source_categories and len(list_source_categories) > 0:
        node['categories'] = list_source_categories

    # return
    return node

In [19]:
# __main__
# query genetics kp
nodes = get_nodes_one_hop(url=url_genepro, list_source=[curie_disease], list_target=[], list_source_categories=[type_disease], list_target_categories=[type_gene], list_predicates=[type_predicate], log=True)

build trapi payload: 
{
    "message": {
        "query_graph": {
            "edges": {
                "e00": {
                    "subject": "n00",
                    "object": "n01",
                    "predicates": [
                        "biolink:related_to"
                    ]
                }
            },
            "nodes": {
                "n00": {
                    "ids": [
                        "MONDO:0004975"
                    ],
                    "categories": [
                        "biolink:Disease"
                    ]
                },
                "n01": {
                    "categories": [
                        "biolink:Gene"
                    ]
                }
            }
        }
    }
}
query: https://translator.broadinstitute.org/genetics_provider/trapi/v1.3/query
got results from: https://translator.broadinstitute.org/genetics_provider/trapi/v1.3/query
got 423 resulting nodes: [('EFO:1001870', "Late-onset Alzheimer's disease